## 필요한 모듈 및 라이브러리 불러오기

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

from tensorflow.keras import layers, models

SEED = 42
np.random.seed(SEED)      # numpy의 전역 난수 시드를 설정
tf.random.set_seed(SEED)  # Tensorflow의 전역 난수 시드를 정

ImportError: DLL load failed while importing _pywrap_checkpoint_reader: 애플리케이션 제어 정책에서 이 파일을 차단했습니다.

### Synthetic Ransomware Dataset 생성  
실제 악성 코드 바이너리를 다루지 않고, 행위 피처(behavior features)만 통계적으로 모사한 Synthetic 데이터셋 생성.  
--> 실제 환경에서는 EDR/로그 시스템에서 추출되는 Feature Table에 해당하는 개념.

### Feature 설계 개요
- encryption_count : 암호화된 파일 개수
  - Encryptor 계열 랜섬웨어는 파일 암호화가 핵심 동작이기 때문에 수치가 매우 높음.
  - 정상 프로세스트에는 급격히 증가하는 암호화 시도 횟수는 없음
- file_write_rate : 초당 파일 쓰기 속도
  - 암호화 과정에서 파일 overwrite, remane등이 폭증
  - Encryptor, Worm-propagating, Human-operated 계열에서 증가
- registry_modifications : 레지스트리 변경 횟수
  - 지속성(Persistence) 확보를 위해 Run Key 수정, UAC 우회, 서비스 등록 등
- network_connections
  - 네트워크 이벤트 수
  - 예 : C2 서버 연결, lateral movement 시도, 네트워크 스캔 등
  - Worm, Exploit 기반 랜섬웨어는 내부 네트워크로 퍼지기 때문에 값이 높음
- process_tree_depth : 프로세스 트리 깊이
  - 공격 도구가 여러 단계를 거쳐 실행되면 깊이가 늘어남
  - Human-operated의 경우 공격자가 다양한 실행 단계를 밟기 때문에 값이 높음
- bruteforce_attempts : 비밀번호 추측 공격(RDP 등) 횟수(브루트포스 시도 횟수)
  - RDP brute-force 공격 기반 랜섬웨어 (예: Dharma, SamSam)는 값이 매우 높음
  - Encryptor/Locker 계열은 brue-force 단계가 거의 없음
- usb_access_count : USB 접근 횟수
  - USB 매체를 통해 전파되는 유형은 값이 매우 높음
  - 기업 내부 오프라인 확산을 노리는 공격에서 두드러짐
- cloud_api_access : 클라우드 API 호출 횟수
  - 최근 랜섬웨어는 OneDrive/Google Drive 등 클라우드 스토리치 파일도 암호화함
  - Cloud-targeted 랜섬웨어는 높은 수치
- phishing_indicator : 피싱 기반의 초기 유입 여부(0~1 사이 확률)
  - 이메일 첨부파일/링크 기반 공격은 이 값이 높도록 설정함
  - Phshing-based Ransom 유형은 거의 1에 가까움
- exploit_indicator : 취약점(Exploit) 기반 침입 여부(0~1 사이 확률)
  - EternalBlue 같은 익스플로잇 활용 공격은 값이 높음
  - Exploit-based Ransom 유형에서 특히 높게 설정

### 1. 랜섬웨어 Synthetic 데이터셋 생성

In [ ]:
# ==================================================
# 클래스 ID를 랜섬웨어 유형 이름으로 매핑하는 딕셔너리 정의
# ==================================================
class_names = {
    0: "Encryptor",
    1: "Locker",
    2: "Wiper",
    3: "Worm-propagating Ransom",
    4: "Human-operated Ransom",
    5: "Phishing-based Ransom",
    6: "RDP Brute-force based",
    7: "Exploit-based Ransom",
    8: "USB/Removable-media Ransom",
    9: "Cloud/SaaS-targeted Ransom",
}


# ============================================================
# 각 랜섬웨어 유형별 평균적인 행위 특징(Behavior Feature 값)을 정의
# 각 class_id 별로 해당 행동 패턴의 특징값을 params에 넣은 매칭 구조
# ============================================================
params = {
    0: dict(enc=800,  fw=300,  reg=80,  net=40,  depth=7,  brute=5,  usb=5,  cloud=5,  ph=0.1, ex=0.2),
    1: dict(enc=5,    fw=20,   reg=10,  net=10,  depth=4,  brute=2,  usb=3,  cloud=3,  ph=0.0, ex=0.1),
    2: dict(enc=10,   fw=150,  reg=30,  net=20,  depth=5,  brute=3,  usb=4,  cloud=4,  ph=0.0, ex=0.2),
    3: dict(enc=400,  fw=250,  reg=70,  net=80,  depth=8,  brute=6,  usb=6,  cloud=6,  ph=0.1, ex=0.3),
    4: dict(enc=900,  fw=350,  reg=120, net=60,  depth=9,  brute=8,  usb=5,  cloud=5,  ph=0.2, ex=0.3),
    5: dict(enc=300,  fw=150,  reg=40,  net=30,  depth=5,  brute=3,  usb=3,  cloud=3,  ph=0.9, ex=0.2),
    6: dict(enc=200,  fw=180,  reg=60,  net=50,  depth=6,  brute=20, usb=4,  cloud=4,  ph=0.1, ex=0.2),
    7: dict(enc=500,  fw=220,  reg=70,  net=70,  depth=7,  brute=6,  usb=4,  cloud=4,  ph=0.2, ex=0.9),
    8: dict(enc=250,  fw=200,  reg=50,  net=25,  depth=5,  brute=4,  usb=30, cloud=3,  ph=0.1, ex=0.2),
    9: dict(enc=350,  fw=220,  reg=60,  net=35,  depth=6,  brute=4,  usb=4,  cloud=25, ph=0.2, ex=0.3),
}


# =========================================
# 전체 데이터에서 각 클래스가 등장한 비율을 정의
# encryptor는 전체에서 15%가 되도록 샘플링
# 0: "Encryptor",
# 1: "Locker",
# 2: "Wiper",
# 3: "Worm-propagating Ransom",
# 4: "Human-operated Ransom",
# 5: "Phishing-based Ransom",
# 6: "RDP Brute-force based",
# 7: "Exploit-based Ransom",
# 8: "USB/Removable-media Ransom",
# 9: "Cloud/SaaS-targeted Ransom",
# =========================================
class_probs = [0.15, 0.05, 0.05, 0.10, 0.15, 0.10, 0.10, 0.10, 0.10, 0.10]


# ==========================================
# Synthetic Ransomware Dataset 생성 함수 정의
# ==========================================
def generate_synthetic_ransomware(n_samples: int = 10000, seed: int = 42) -> pd.DataFrame:
    
    np.random.seed(seed) # numpy 난수 시드를 고정
    
    rows = []            # 각 샘플 정보를 저장할 리스트를 초기화

    # for문으로 지정된 샘플 수만큼 반복하며 데이터를 생성
    for i in range(n_samples):
        # class_probs 분포에 따라 클래스 ID를 하나 샘플링
        c = np.random.choice(list(class_names.keys()), p=class_probs)   # class_names.keys() -> [0,1,2,3,4,5,6,7,8,9]
        
        # 선택된 클래스의 평균 파라미터를 가져옴
        p = params[c]

        # 정규 분포를 사용하여 연속형 피처들을 생성
        enc   = max(0, np.random.normal(p["enc"],  p["enc"]  * 0.3))
        fw    = max(0, np.random.normal(p["fw"],   p["fw"]   * 0.3))
        reg   = max(0, np.random.normal(p["reg"],  max(5, p["reg"] * 0.3)))
        net   = max(0, np.random.normal(p["net"],  max(5, p["net"] * 0.3)))
        depth = max(1, np.random.normal(p["depth"], 1.0))
        brute = max(0, np.random.normal(p["brute"], max(1, p["brute"] * 0.4)))
        usb   = max(0, np.random.normal(p["usb"],  max(1, p["usb"]  * 0.5)))
        cloud = max(0, np.random.normal(p["cloud"],max(1, p["cloud"]* 0.5)))

        # 피싱/익스플로잇 여부는 Bernoulli 분포로 생성
        ph_ind = np.random.rand() < p["ph"]
        ex_ind = np.random.rand() < p["ex"]

        # 한 샘플의 모든 정보를 딕셔너리에 담아 리스트에 추가
        rows.append({
            "sample_id": f"sample_{i:05d}",
            "encryption_count": int(enc),
            "file_write_rate": fw,
            "registry_modifications": int(reg),
            "network_connections": int(net),
            "process_tree_depth": int(depth),
            "bruteforce_attempts": int(brute),
            "usb_access_count": int(usb),
            "cloud_api_access": int(cloud),
            "phishing_indicator": int(ph_ind),
            "exploit_indicator": int(ex_ind),
            "class_id": c,
            "class_name": class_names[c],
        })
    # 설명: 리스트를 pandas DataFrame으로 변환하여 반환
    df = pd.DataFrame(rows)
    return df


# 위 함수로 10,000개 샘플 생성
df = generate_synthetic_ransomware(n_samples=10000, seed=SEED)

print(" Synthetic Ransomware Dataset 생성 완료")
df.head()

## EDA - 데이터 구조 및 분포 파악
기본 통계량, 클래스 분포, 피처 상관 관계 등을 확인하여 **랜섬웨어 유형과 행위 특징의 연관성**을 시각적으로 파악

In [ ]:
# ==========================
# 데이터 구조 및 기본 통계 확인
# ==========================

# DataFrame의 정보
print("[INFO] DataFrame info")
print(df.info())

# 숫자형 컬럼들에 대한 기초 통계량 출력
print("\n[INFO] Numeric Describe")
print(df.describe())

In [ ]:
# ================
# 클래스 분포 시각화
# ================

# class_name 기준으로 각 랜섬웨어 유형의 개수 계산
class_counts = df["class_name"].value_counts()

# 클래스 분포를 막대그래프로 그리기
plt.figure(figsize=(8, 4))
plt.bar(class_counts.index, class_counts.values)
plt.xticks(rotation=45, ha="right")
plt.title("Class Distribution (Ransomware Types)")
plt.xlabel("Ransomware Type")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

# 숫자 값으로 클래스 분포 확인
print("\n[INFO] Class Distribution")
print(class_counts)


### 클래스 분포 해석

- Encryptor와 Human-operated Ransom 비중이 상대적으로 높게 나타나도록 설계됨
- USB/Cloud 기반 등의 클래스도 일정 비율을 차지해 멀티클래스 학습에 적합함
- 실제 환경에서는 회사별 위협 프로파일에 따라 이 분포가 달라지며,
  희귀 클래스에 대해서는 Oversampling 또는 비용 민감 학습이 필요할 수 있음

In [ ]:
# ==================
# 피처 상관관계 히트맵
# ==================

# 상관계수를 계산할 숫자형 피처 목록 정의
numeric_cols = [
    "encryption_count",
    "file_write_rate",
    "registry_modifications",
    "network_connections",
    "process_tree_depth",
    "bruteforce_attempts",
    "usb_access_count",
    "cloud_api_access",
    "phishing_indicator",
    "exploit_indicator",
]

# 상관계수 행렬 계산
corr = df[numeric_cols].corr()

# 히트맵 형태로 시각화
plt.figure(figsize=(8, 6))
im = plt.imshow(corr, cmap="coolwarm", interpolation="nearest")
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.xticks(range(len(numeric_cols)), numeric_cols, rotation=45, ha="right")
plt.yticks(range(len(numeric_cols)), numeric_cols)
plt.title("Feature Correlation Heatmap")
plt.tight_layout()
plt.show()

In [ ]:
# ===============================================================================
# 클래스별 encryption_count 분포 (Boxplot)  
#  - 파일 암호화 시도 횟수(랜섬웨어 유형을 구분하는 가장 중요한 지표)
#    >> 랜섬웨어 유형별 주요 행동 패턴 차이 확인
#    >> 비정상/변종 탐지의 기준값을 잡을 수 있음
#       - Encryptor인데 encryption_count가 갑자기 낮다면 → 변종, 방해, 실패 가능성
#       - Wiper인데 encryption_count가 너무 높으면 → 정상 패턴 벗어남(anomaly)
# ===============================================================================

# 클래스별 encryption_count 값을 리스트로 모음
groups = []
labels = []
for cid, cname in class_names.items():
    groups.append(df[df["class_id"] == cid]["encryption_count"].values)
    labels.append(cname)

# Boxplot을 그려 클래스별 암호화 개수 분포 비교
plt.figure(figsize=(10, 5))
plt.boxplot(groups, labels=labels, showfliers=False)
plt.xticks(rotation=45, ha="right")
plt.ylabel("encryption_count")
plt.title("Encryption Count Distribution by Class")
plt.tight_layout()
plt.show()

### Encryption Count Boxplot 해석

- Encryptor, Human-operated Ransom 유형에서 encryption_count의 중앙값과 상단 사분위가 높게 나타남
- 중앙값(Median) : 각 랜섬웨어 유형이 평균적으로 얼마나 많은 파일을 암호화하는지
- IQR(Interquartile Range) : 암호화 수치의 변동성이 얼마나 큰지
- 이런 패턴은 각 랜섬웨어 유형의 "행위 시그니처"를 파악하는 데 도움이 됨
- Encryptor 계열(클래스 0)은 암호화 개수가 매우 높음
- Locker(1), Wiper(2) 같은 유형은 encryption_count가 거의 없음
- Worm-propagating, Human-operated Ransom은 보통 높은 암호화와 함께 다른 공격 요소를 수행함
- USB 기반(8번)이나 Exploit 기반(7번)은 비교적 중간 수준임

---

## 머신러닝 - RandomForest 기반 멀티 클래스 분류


In [ ]:
# ==============================
# Feature / Target 분리 및 전처리
# ==============================

# 모델 입력 피처 리스트 정의
feature_cols = [
    "encryption_count",
    "file_write_rate",
    "registry_modifications",
    "network_connections",
    "process_tree_depth",
    "bruteforce_attempts",
    "usb_access_count",
    "cloud_api_access",
    "phishing_indicator",
    "exploit_indicator"
]


# 입력 피처 행렬 x와 타깃 레이블 y를 정의
X = df[feature_cols].values
y = df["class_id"].values


# train_test_split을 이용해 데이터를 학습용과 테스트용으로 나눔
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=SEED,
    stratify=y
)


# 피처 스케일링을 위한 StandardScaler를 초기화
scaler = StandardScaler()

# 학습 데이터에 대해 스케일러를 학습시키고 변환
X_train_scaled = scaler.fit_transform(X_train)

# 학습된 스케일러를 사용해 테스트 데이터도 변환
X_test_scaled = scaler.transform(X_test)

# 분리된 Train/Test 데이터의 크기 출력
print("Train shape: ", X_train.shape, "Test shape: ", X_test.shape)


---

## LSTM 기반 랜섬웨어 유형 분류 (시퀀스 모델)
실제 보안 로그는 시간에 따른 이벤트 시퀀스로 구성됨
- Tabular Feature를 5 타임 스텝으로 반복한 pseudo-sequence를 만들어 LSTM 구조를 테스트

In [ ]:
# ===================================
# LSTM 입력 데이터 준비(Pseudo-Squence)
# ===================================

# 시퀀스 길이(타임스텝 수)를 5로 설정
timesteps = 5

# 학습용 데이터를 배치, 타임스텝, 피처수 형태로 변환
X_train_seq = np.repeat(X_train_scaled[:, np.newaxis, :], timesteps, axis=1)

# 테스트용 데이터도 동일한 방식으로 변환
X_test_seq = np.repeat(X_test_scaled[:, np.newaxis, :], timesteps, axis=1)

# 변환된 LSTM 입력 데이터의 형태를 출력
print("LSTM Input Shape - Train: ", X_train_seq.shape)
print("LSTM Input Shape - Test: ", X_test_seq.shape)

### ***LSTM 분류 모델 정의***

In [ ]:
# LSTM 기반 분류 모델을 생성하는 함수 정의
def build_lstm_classifier(input_shape, num_classes:int):
    # Sequential API를 사용해 레이어를 순차적으로 stack
    model = models.Sequential()

    # 입력 형태를 지정하는 Input 레이어
    model.add(layers.Input(shape=input_shape))

    # 시퀀스 데이터를 처리할 LSTM 레이어
    model.add(layers.LSTM(64))

    # 은닉층으로 64차원 Dense 레이어 추가
    model.add(layers.Dense(64, activation="relu"))

    # 출력 레이어로 클래스 개수 만큼 뉴런을 두고 softmax로 확률 출력
    model.add(layers.Dense(num_classes, activation="softmax"))

    # 모델을 컴파일하면서 손실 함수와 최적화 알고리즘, 평가 지표 지정
    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    
    return model


# LSTM 모델 생성
lstm_model = build_lstm_classifier(
    input_shape = (timesteps, X_train_scaled.shape[1]),
    num_classes = len(class_names)
)


# LSTM 모델 구조 요약 정보 출력
lstm_model.summary()

### ***LSTM 모델 학습 및 학습 곡선 시각화***

In [ ]:
# LSTM 모델을 학습 데이터에 학습
history = lstm_model.fit(
    X_train_seq,            # 시퀀스 형태의 학습 입력 데이터
    y_train,                # 정답 class_id 레이블
    epochs=5,               # 에폭 수는 데모를 위해 5로 설정
    batch_size =64,         # 배치 크기
    validation_split =0.2,  # 학습 데이터의 20%를 검증용으로 사용
    verbose=1               # 학습 과정을 로그로 출력
)


# history 객체에 저장된 학습/검증 손실 및 정확도 곡선 가져옴
history_dict = history.history

# 두 개의 서브플롯(손실, 정확도)을 그리기 위해 그림 크기 설정
plt.figure(figsize=(10, 4))

# 첫 번째 서브 프롨에 학습/검증 "손실" 곡선 그리기
plt.subplot(1, 2, 1)
plt.plot(history_dict["loss"], label="train_loss")
plt.plot(history_dict["val_loss"], label="val_loss")
plt.title("LSTM Training/Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()


# 두 번ㅌ째 서브 플롯에 학습/검증 "정확도" 곡선 그리기
plt.subplot(1, 2, 2)
plt.plot(history_dict["accuracy"], label="train_acc")
plt.plot(history_dict["val_accuracy"], label="val_acc")
plt.title("LSTM Training/Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()


# 두 서브 플롯 사이의 레이아웃을 자동 조정
plt.tight_layout()
plt.show()

# 학습이 완료된 LSTM 모델을 테스트 세트에서 평가
loss_lstm, acc_lstm = lstm_model.evaluate(X_test_seq, y_test, verbose=0)

print(f"[LSTM] Test Loss: {loss_lstm:.4f}, Test Accuracy: {acc_lstm:.4f}")

### LSTM 학습 해석

- Loss 곡선이 안정적으로 감소하고, Train/Validation 간 큰 차이가 없으면 과적합이 심하지 않다고 볼 수 있습니다.
- Accuracy 곡선이 일정 수준 이상에서 수렴하면 모델이 데이터의 패턴을 어느 정도 학습한 것으로 해석할 수 있습니다.
- 실제 환경에서 시퀀스 로그(시간 순서 이벤트)를 입력으로 사용하면 LSTM/Transformer의 장점을 더 활용할 수 있습니다.

### ***LSTM 단일 샘플 예측 예시***

In [ ]:
# 테스트 세트에서 첫 번째 시퀀스 샘플 선택
sample_idx = 2

# 선택한 샘플을 LSTM 모델에 입력하여 클래스별 확률 예측
lstm_probs = lstm_model.predict(X_test_seq[sample_idx:sample_idx+1])

# 가장 높은 확률을 가진 인덱스를 예측 클래스 ID로 선택
lstm_pred_class = int(np.argmax(lstm_probs))

# 실제 정답과 예측 결과 출력
print("=== LSTM 단일 샘플 예측 예시 ===")
print("Sample Index: ", sample_idx)
print("True Class  : ", class_names[y_test[sample_idx]])
print("Predicted Class: ", class_names[lstm_pred_class])

# 각 클래스에 대한 예측 확률 분포도 함께 출력
print("Class probabilities: ", [f"{p:.10f}" for p in lstm_probs[0]])

---

## Autoencoder 기반 Encryptor 이상 탐지
Encryptor(class_id==0)를 "정상 패턴"으로 간주하고 Autoencoder로 해당 패턴을 학습 이후 재구성 오차(MSE)를 활용해 이상 여부를 판단하는 이상탐지 결과 확인